This notebook allow you to print results of models in tables

In [1]:
# from twinews import config as twinewsConf
# twinewsConf.mongoLocation = "octods"

In [2]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from twinews.utils import *
from twinews.evaluation.utils import *
from twinews.models.ranking import *

In [3]:
if False:
    purgeSubsampledRankings()

In [4]:
if False:
    removeRankingsAndScores('tfidf-b0e81')

In [5]:
def basicPrint(model, *args, b=None, w=None, splitVersion=2, **kwargs):
    if w is not None and isinstance(w, str):
        w = [w]
    if b is not None and isinstance(b, str):
        b = [b]
    if w is not None and len(args) > 0:
        args = list(args) + list(w)
    printReportKwargs = \
    {
        'model': model,
        'splitVersion': splitVersion,
        'noSubsampling': True,
        'metaFilter': \
        {
            # "implementation": "gensim-lda",
        },
        # 'allowedMetrics': {'ndcg', 'div@100', 'topic-div@100', 'swjacc-div@100', 'jacc-div@100'}, # None, {'ndcg', 'ndcg@10'}
        'discardedMetrics': {'map', 'mrr', 'ndcg@10', 'ndcg@100', 'p@10', 'p@100'},
        'whiteMetricPatterns': None if len(args) == 0 else set(list(args) + ['ndcg']),
        'blackMetricPatterns': b,
        'sortBy': 'ndcg',
    }
    printReportKwargs = mergeDicts(printReportKwargs, kwargs)
    printReport(**printReportKwargs)

In [6]:
def printAll(*args, **kwargs):
    kwargs['onlyFields'] = {'id', 'model'}
    kwargs['excludedModels'] = {'combin', 'ideal', 'worst', 'random'}
    return basicPrint(None, *args, **kwargs)

# Misc

In [7]:
if False:
    twinewsRankings = getTwinewsRankings()
    keysToDelete = set()
    for key in twinewsRankings.keys():
        if "combin" in key:
            meta = twinewsRankings.getMeta(key)
            hasTFIDF = False
            for m in meta['models']:
                if 'tfidf' in m:
                    hasTFIDF = True
                    break
            if hasTFIDF and meta['splitVersion'] == 1:
                keysToDelete.add(key)
    print(keysToDelete)

In [8]:
if False:
    for key in keysToDelete:
        removeRankingsAndScores(key)

In [9]:
# Removing combin models with maxUsers is not None:
if False:
    twinewsRankings = getTwinewsRankings()
    ids = twinewsRankings.keys()
    bp(ids)
    for currentId in ids:
        if 'combin' in currentId:
            meta = twinewsRankings.getMeta(currentId)
            if meta['maxUsers'] is not None:
                del twinewsRankings[currentId]

# Model lists per split version

In [ ]:
def printModelList(splitVersion):
    twinewsRankings = getTwinewsRankings()
    models = set()
    for key in twinewsRankings.keys():
        meta = twinewsRankings.getMeta(key)
        if meta["splitVersion"] == splitVersion:
            models.add(meta['model'])
    print(" ".join(sorted(list(models))) + " (" + str(len(models)) + ")")

In [ ]:
printModelList(2)

In [ ]:
printModelList(1)

# Reference models

In [ ]:
basicPrint('ideal', splitVersion=2)
basicPrint('random', splitVersion=2)
basicPrint('worst', splitVersion=2)

# Ad-hoc models

In [ ]:
basicPrint('jaccard', 'div', b=['topic', 'jacc', 'style'])

In [ ]:
basicPrint('bm25')

In [10]:
basicPrint('BM25_yf', noSubsampling=False)

Found key difference: {'empiricalParameter'}
These values are common to all rows (4):

	- splitVersion: 2
	- epsilon: 0.25
	- maxDocuments: 10000
	- minDF: 0.0005
	- model: BM25_yf
	- lowercase: False
	- doLemmatization: True
	- implementation: BM25
	- maxDF: 300
	- useExtraNews: False
Normalized metrics: {'nov@100', 'snov@100', 'div@100', 'topic-snov@100', 'topic-nov@100', 'topic-div@100', 'style-div@100', 'style-nov@100', 'style-snov@100'}
serializabledict twinews-dominances initialised.


,id,b,empiricalParameter,k1,maxUsers,ndcg,div@100,topic-div@100,jacc-div@100,swjacc-div@100,style-div@100,nov@100,topic-nov@100,jacc-nov@100,swjacc-nov@100,style-nov@100,snov@100,topic-snov@100,jacc-snov@100,swjacc-snov@100,style-snov@100,tfidf-ser@100,wtfidf-ser@100,bm25-ser@100,jacc-ser@100,style-ser@100
3,BM25_yf-78651,0.75,N/A,1.8,30,0.5355,0.4256,0.323333,0.86864,0.94611,0.36185,0.4841,0.280767,0.8788,0.95526,0.366625,0.43035,0.37108,0.83402,0.91728,0.38025,0.34401,0.16622,0.15789,0.54668,0.51014
1,BM25_yf-42d08,0.75,True,1.2,30,0.52538,0.4131,0.327467,0.86747,0.94434,0.3553,0.4829,0.291333,0.87927,0.95491,0.366,0.4324,0.38748,0.83365,0.91621,0.3825,0.29273,0.10307,0.15789,0.5184,0.46666
2,BM25_yf-30805,0.75,N/A,1.8,nan,0.51892,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
0,BM25_yf-c91a6,0,N/A,0,nan,0.33871,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


# Models that use vector representations

In [ ]:
basicPrint('lda')

In [ ]:
basicPrint('nmf')

In [ ]:
basicPrint('tfidf')

In [ ]:
basicPrint('dbert-ft')

In [ ]:
basicPrint('dbert-base')

In [ ]:
basicPrint('stylo')

In [ ]:
basicPrint('infersent')

In [ ]:
basicPrint('sent2vec')

In [ ]:
basicPrint('word2vec')

In [ ]:
basicPrint('doc2vec')

In [ ]:
basicPrint('usent')

In [ ]:
basicPrint('bert')

# All models for split version 2

In [ ]:
printAll('div', splitVersion=2)

# Combinations of models

In [ ]:
# Expected 156 rows:
basicPrint('combin')

# Deep Learning models

In [ ]:
basicPrint('DSSM_yf', noSubsampling=False)

# Reports on the test set (split version 1)

In [ ]:
basicPrint('ideal', splitVersion=1)
basicPrint('random', splitVersion=1)
basicPrint('worst', splitVersion=1)

In [ ]:
printAll('div', splitVersion=1, doNormalization=True)

In [ ]:
basicPrint('combin', 'nov', splitVersion=1)